In [1]:
import json

import h5py
import numpy as np
import pandas
from keras.layers import Dense, ZeroPadding2D, Convolution2D, MaxPooling2D, Dropout
from keras.layers import Flatten
from keras.models import Sequential
from keras.optimizers import Adam
import scipy
from scipy.misc import imresize
from scipy.ndimage import imread
from scipy.misc import toimage
import cv2, numpy as np
from PIL import Image

img_width = 80
img_height = 40

Using TensorFlow backend.


In [2]:
#from socketIO_client import SocketIO, LoggingNamespace

In [3]:
import socketio

In [4]:
data = pandas.read_csv(
        'D:/Self Driving Car Engineer/P3/data/driving_log.csv',
        names=['center', 'left', 'right', 'steering', 'throttle', 'brake', 'speed'])

In [5]:
data['steering'].values[1]

' 0'

In [6]:
print(data['center'])

0                                       center
1       IMG/center_2016_12_01_13_30_48_287.jpg
2       IMG/center_2016_12_01_13_30_48_404.jpg
3       IMG/center_2016_12_01_13_31_12_937.jpg
4       IMG/center_2016_12_01_13_31_13_037.jpg
5       IMG/center_2016_12_01_13_31_13_177.jpg
6       IMG/center_2016_12_01_13_31_13_279.jpg
7       IMG/center_2016_12_01_13_31_13_381.jpg
8       IMG/center_2016_12_01_13_31_13_482.jpg
9       IMG/center_2016_12_01_13_31_13_584.jpg
10      IMG/center_2016_12_01_13_31_13_686.jpg
11      IMG/center_2016_12_01_13_31_13_786.jpg
12      IMG/center_2016_12_01_13_31_13_890.jpg
13      IMG/center_2016_12_01_13_31_13_991.jpg
14      IMG/center_2016_12_01_13_31_14_092.jpg
15      IMG/center_2016_12_01_13_31_14_194.jpg
16      IMG/center_2016_12_01_13_31_14_295.jpg
17      IMG/center_2016_12_01_13_31_14_398.jpg
18      IMG/center_2016_12_01_13_31_14_500.jpg
19      IMG/center_2016_12_01_13_31_14_602.jpg
20      IMG/center_2016_12_01_13_31_14_702.jpg
21      IMG/c

In [18]:
-(data['steering'].values[1:])

TypeError: bad operand type for unary -: 'str'

In [28]:
#-(data['steering'].values[1:])

In [ ]:
data['steering'].values[::-1] #only flips the array

In [9]:
data['steering'].values

array(['steering', ' 0', ' 0', ..., ' 0', ' 0', ' 0'], dtype=object)

In [35]:
norm_steering = (data['steering'].values[1:]) #this gets rid of the first cell which is 'steering'


In [36]:
flip_steering = norm_steering[::-1] #flip the data
print(flip_steering)

[' 0' ' 0' ' 0' ..., ' 0' ' 0' ' 0']


In [37]:
data['steering'].values[1:]

array([' 0', ' 0', ' 0', ..., ' 0', ' 0', ' 0'], dtype=object)

In [44]:

def load_data():
    data = pandas.read_csv(
        'D:/Self Driving Car Engineer/P3/data/driving_log.csv',
        names=['center', 'left', 'right', 'steering', 'throttle', 'brake', 'speed']
    )

    X_train = np.transpose(
        np.array([imresize(imread(fname), (img_height, img_width)) for fname in data['center'].values[1:]]), (0, 3, 2, 1)
    ) / 255.0
    y_train = data['steering'].values[1:]

    X_train_flip = np.transpose(
        np.array([np.fliplr(imresize(imread(fname), (img_height, img_width))) for fname in data['center'].values[1:]]), (0, 3, 2, 1)
    ) / 255.0
    #y_train_flip = -(data['steering'].values[1:])
    #y_train_flip = data['steering'].values
    y_train_flip = norm_steering[::-1]
    #y_train_flip = -1*(data['steering'].values[1:])

    #return np.concatenate((X_train, X_train_flip)), np.concatenate((y_train, y_train_flip))
    return X_train, y_train


In [45]:
def load_weights(model):
    f = h5py.File('D:/Self Driving Car Engineer/P3/data/vgg19_weights.h5')
    for k in range(f.attrs['nb_layers']):
        if k >= len(model.layers):
            # we don't look at the last (fully-connected) layers in the savefile
            break
        g = f['layer_{}'.format(k)]
        weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]
        model.layers[k].set_weights(weights)
    f.close()

In [46]:
def SmallNetwork():
    model = Sequential()
    model.add(ZeroPadding2D((1, 1), input_shape=(3, img_width, img_height)))
    model.add(Convolution2D(32, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(32, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    model.add(Flatten(input_shape=input_shape))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))

    return model

In [47]:
def VGG_19(weights_path=None):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(3,224,224)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation='softmax'))

    if weights_path:
        model.load_weights(weights_path)

    return model

In [48]:
'''if __name__ == "__main__":
    im = cv2.resize(cv2.imread('cat.jpg'), (224, 224)).astype(np.float32)
    im[:,:,0] -= 103.939
    im[:,:,1] -= 116.779
    im[:,:,2] -= 123.68
    im = im.transpose((2,0,1))
    im = np.expand_dims(im, axis=0)

    # Test pretrained model
    model = VGG_19('vgg19_weights.h5')
    sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy')
    out = model.predict(im)
    print np.argmax(out)'''

'if __name__ == "__main__":\n    im = cv2.resize(cv2.imread(\'cat.jpg\'), (224, 224)).astype(np.float32)\n    im[:,:,0] -= 103.939\n    im[:,:,1] -= 116.779\n    im[:,:,2] -= 123.68\n    im = im.transpose((2,0,1))\n    im = np.expand_dims(im, axis=0)\n\n    # Test pretrained model\n    model = VGG_19(\'vgg19_weights.h5\')\n    sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)\n    model.compile(optimizer=sgd, loss=\'categorical_crossentropy\')\n    out = model.predict(im)\n    print np.argmax(out)'

In [49]:
if __name__ == '__main__':
    X_train, y_train = load_data()

    input_shape = X_train.shape[1:]
    n_classes = len(np.unique(y_train))

    model = SmallNetwork()

    opt = Adam()
    model.compile(loss='mse', optimizer=opt, metrics=['accuracy'])
    model.fit(X_train, y_train, batch_size=512, nb_epoch=10) #10 epochs?

    with open('model.json', 'w') as fd:
        json.dump(model.to_json(), fd)

    model.save_weights('C:/CarND-Transfer-Learning-Lab/model.h5')

Epoch 1/10
8036/8036 [==============================] - 12s - loss: 0.0170 - acc: 0.5427    
Epoch 2/10
8036/8036 [==============================] - 11s - loss: 0.0158 - acc: 0.5427    
Epoch 3/10
8036/8036 [==============================] - 11s - loss: 0.0143 - acc: 0.5427    
Epoch 4/10
8036/8036 [==============================] - 11s - loss: 0.0128 - acc: 0.5427    
Epoch 5/10
8036/8036 [==============================] - 11s - loss: 0.0130 - acc: 0.5427    
Epoch 6/10
8036/8036 [==============================] - 11s - loss: 0.0116 - acc: 0.5427    
Epoch 7/10
8036/8036 [==============================] - 11s - loss: 0.0107 - acc: 0.5427    
Epoch 8/10
8036/8036 [==============================] - 11s - loss: 0.0105 - acc: 0.5427    
Epoch 9/10
8036/8036 [==============================] - 11s - loss: 0.0101 - acc: 0.5427    
Epoch 10/10
8036/8036 [==============================] - 11s - loss: 0.0099 - acc: 0.5427    
